In [ ]:
import pyspark
from pyspark.sql import SQLContext

In [ ]:
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [ ]:
sc

In [ ]:
WARCs_PATH="*.warc.gz"

In [ ]:
from pyspark.sql.functions import udf

url_correction = udf(lambda s: s[1:-1] if len(s) > 0 and s[0] == '<' and s[-1] == '>' else s)

In [ ]:
# Temporal folder for CSV files containing partial results
OUTPUT_PATH="full-links-all/"

In [ ]:
from aut import *
from pyspark.sql.functions import desc, col

WebArchive(sc, sqlContext, WARCs_PATH) \
  .webpages() \
  .withColumn("url", url_correction("url")) \
  .select("url") \
  .select(remove_prefix_www(extract_domain(col("url"))).alias("domain"), 'url') \
  .groupBy("domain", "url") \
  .count() \
  .sort(desc("count")) \
  .write.csv(OUTPUT_PATH)


In [ ]:
# Create a single CSV with partial results 
!cat "$OUTPUT_PATH"/*.csv > full-links-all.csv

In [ ]:
# Delete temporal folder
!rm -r "$OUTPUT_PATH"